# Notebook for exporting screenshots for annotation

In [38]:
import os
import random
import json
import shutil

from ultralytics import YOLO
from PIL import Image

import sys
sys.path.insert(0, "../../hindsight/hindsight_server/")

from db import HindsightDB
from utils import make_dir

In [27]:
db = HindsightDB()

# If selecting frames from Hindsight

In [13]:
frames = db.get_frames(impute_applications=False)
frames = frames.loc[frames['application'] == "Twitter"]

In [14]:
sel_frames = frames.sample(n=100)

In [15]:
dest_dir = "../data/annotations/twitter/rand_upload_2/"
make_dir(dest_dir)
for f in sel_frames['path']:
    dest_f = os.path.join(dest_dir, os.path.basename(f))
    shutil.copy(f, dest_dir)

# Run prediction model on frames

In [29]:
# trained_model = YOLO("./runs/detect/train15/weights/best.pt")
trained_model = YOLO("/Users/connorparish/code/hindsight_parsing/data/runs/detect/tweet_parse-2024-09-17-14-15-b4e9db47/weights/best.pt")

In [34]:
dest_dir = "/Users/connorparish/code/hindsight_parsing/data/extracted_entities/tweets/complete_unique"

In [35]:
pred_images_dir = dest_dir 
images = list()
for f in os.listdir(pred_images_dir):
    im = Image.open(os.path.join(pred_images_dir, f))
    images.append(im)

In [39]:
images = random.sample(images, 100)

In [41]:
results = trained_model(images)


0: 640x640 1 image_content_source, 1 impresssions, 1 likes, 1 replies, 2 retweetss, 1 time_since_post, 1 tweet_image_content, 1 tweet_text, 1 user_handle, 1 user_image, 1 username, 1 verified_check, 263.1ms
1: 640x640 1 impresssions, 1 likes, 1 replies, 1 retweets, 1 time_since_post, 1 tweet_image_content, 1 tweet_text, 1 user_handle, 1 user_image, 1 username, 1 verified_check, 263.1ms
2: 640x640 1 impresssions, 1 likes, 2 repliess, 1 retweets, 1 time_since_post, 1 tweet_text, 1 user_handle, 1 user_image, 1 username, 1 verified_check, 263.1ms
3: 640x640 1 impresssions, 1 likes, 1 replies, 1 retweets, 1 time_since_post, 1 tweet_image_content, 1 tweet_text, 1 user_handle, 1 user_image, 3 usernames, 263.1ms
4: 640x640 1 impresssions, 1 likes, 1 replies, 1 retweets, 1 time_since_post, 1 tweet_image_content, 2 tweet_texts, 1 user_handle, 1 user_image, 1 username, 1 verified_check, 263.1ms
5: 640x640 1 impresssions, 1 likes, 1 replies, 1 retweets, 1 time_since_post, 1 tweet_image_content, 1

In [49]:
result = results[1]

In [50]:
result.show()

In [44]:
all_preds = list()

In [46]:
for result in results:
    image_preds_d = {}
    image_path = f"/data/local-files/?d={result.path}"
    image_preds_d['data'] = {"image" : image_path}
    predictions_d = {"model_version": "train13", "score": 0.55}
    org_width = result.orig_shape[1]
    org_height = result.orig_shape[0]
    result_d_template = {"type": "rectanglelabels",        
            "from_name": "label", "to_name": "image",
            "original_width": org_width, "original_height": org_height,
            "image_rotation": 0}
    converted_results = list()
    for i, box in enumerate(result.boxes):
        result_d = result_d_template.copy()
        result_d['id'] = f"result{i}"
        value_d = {"rotation" : 0,
                "x" : ((float(box.xyxyn[0][0])) * 100),
                "y" : (float(box.xyxyn[0][1])) * 100,
                "width" : ((float(box.xyxyn[0][2]) - float(box.xyxyn[0][0])) * 100), 
                "height": (float(box.xyxyn[0][3]) - float(box.xyxyn[0][1])) * 100, 
                "rectanglelabels": [result.names[int(box.cls[0])]]}
        result_d['value'] = value_d
        converted_results.append(result_d)
        
    predictions_d["result"] = converted_results
    image_preds_d['predictions'] = [predictions_d]
    all_preds.append(image_preds_d)

In [47]:
with open("tweet_parse_pred_1.json", 'w') as outfile:
    json.dump(all_preds, outfile)

In [1]:
import sys
sys.path.insert(0, "../")

from annotation_db import HindsightAnnotationsDB

In [2]:
annotation_db = HindsightAnnotationsDB()

In [24]:
annotations = annotation_db.get_annotations()

In [25]:
annotations

,id,frame_id,x,y,w,h,rotation,label,conf,model_name,model_version
